# Import libraries

In [ ]:
import torch
import torch.nn as nn
from torchvision import  transforms
from torchvision.models import vgg16, VGG16_Weights
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from PIL import Image
import os
import numpy as np

# Database creations using pytorch Dataset 

In [ ]:
class ImageAuthenticityDataset(Dataset):
    """Dataset for image quality assessment."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the CSV file with annotations.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        """Returns the number of samples in the dataset."""
        return len(self.data)

    def getInfo(self, idx):
        """
        Retrieves image Category, Challenge and prompt by index.
        
        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            tuple: A tuple (category, challenge, prompt) where:
                category (str): The category of the image.
                challenge (str): The challenge of the image.
                prompt (str): The prompt of the image.
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        category = self.data.iloc[idx, 5]
        challenge = self.data.iloc[idx, 6]
        prompt = self.data.iloc[idx, 7]

        return category, challenge, prompt
    def __getitem__(self, idx):
        """
        Retrieves an image and its labels by index.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            tuple: A tuple (image, labels) where:
                image (PIL.Image): The image.
                labels (torch.Tensor): Tensor containing quality and authenticity scores.
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(os.getcwd(), self.data.iloc[idx, 3])  # image_path column
        image = Image.open(img_name).convert('RGB')
        authenticity = self.data.iloc[idx, 1]  # Authenticity column
        labels = torch.tensor([authenticity], dtype=torch.float)


        if self.transform:
            image = self.transform(image)

        return image, labels


# Definitions of the models

In [ ]:
class AuthenticityPredictor(nn.Module):
    def __init__(self, freeze_backbone=True):
        super().__init__()
        # Load pre-trained VGG16
        vgg = vgg16(weights=VGG16_Weights.DEFAULT)
        
        # Freeze backbone if requested
        if freeze_backbone:
            for param in vgg.features.parameters():
                param.requires_grad = False
                
        # Extract features up to fc2
        self.features = vgg.features
        self.avgpool = vgg.avgpool
        self.fc1 = vgg.classifier[:-1]  # Up to fc2 (4096 -> 128)
        
        # New regression head
        self.regression_head = nn.Sequential(
            nn.Linear(4096, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1)  # Predict quality and realness
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        features = self.fc1(x)
        predictions = self.regression_head(features)
        return predictions, features

## Setup section

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data transformations for the ImageNet dataset
data_transforms = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

ANNOTATIONS_FILE = 'Dataset/AIGCIQA2023/real_images_annotations.csv'

# Create the dataset
dataset = ImageAuthenticityDataset(csv_file=ANNOTATIONS_FILE, transform=data_transforms)

# Set random seeds for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


# Create data loaders
BATCH_SIZE = 64
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# Create a dictionary containing the data loaders
dataloaders = {
    'train': train_dataloader,
    'val': val_dataloader,
    'test': test_dataloader
}


# Models loading

In [ ]:
BASELINE_MODEL_PATH = 'Models/VGG-16_real_authenticity_finetuned.pth'
NOISY_PRUNED_MODEL_PATH = 'Models/real_authenticity_noise_out_pruned_model.pth'


noisy_pruned_model = AuthenticityPredictor()
noisy_pruned_model.load_state_dict(torch.load(NOISY_PRUNED_MODEL_PATH, weights_only=True))

baseline_model = AuthenticityPredictor()
baseline_model.load_state_dict(torch.load(BASELINE_MODEL_PATH,weights_only=True))

# 1) Sort the importance scores by channel ID and normalize them to sum to 1

In [ ]:
# Set numpy print options
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

# Load the array of arrays
importance_scores = np.load('Ranking_arrays/Dual_scores_obj_x_obj_authenticity_importance_scores.npy', allow_pickle=True)

# Fix the array creation with proper generator expressions
delta_residuals_scores = np.array([pair[0] for pair in importance_scores])
delta_predictions_scores = np.array([pair[1] for pair in importance_scores])

def extract_sign(scores):
    """Extract the sign of the scores."""
    return np.sign(scores)


def min_max_scale(scores):
    """Min-max scale the scores to the range [0, 1]."""
    min_val = np.min(scores)
    max_val = np.max(scores)
    # Check for division by zero
    if max_val == min_val:
        return np.zeros_like(scores)
    return (scores - min_val) / (max_val - min_val)

def transform_scores(scores):
    """Transform the scores using the specified transformations."""
    # Remove negative values
    # Normalize the scores to the range [0, 1]
    scores = min_max_scale(scores)
    return scores

# # Initialize array with same structure but with zeros
transformed_importance_scores = np.empty_like(importance_scores, dtype=object)

# Apply the transformation to each importance score array
for i in range(len(delta_predictions_scores)):
    # Convert to numpy array if it's not already (for object arrays)
    if not isinstance(delta_predictions_scores[i], np.ndarray):
        delta_predictions_scores[i] = np.array(delta_predictions_scores[i])
    transformed_importance_scores[i] = transform_scores(delta_predictions_scores[i]) * extract_sign(delta_residuals_scores[i])


# distribution of the transformed importance scores
import matplotlib.pyplot as plt
import seaborn as sns

# Flatten all importance scores into a single array
all_scores = np.concatenate([score.flatten() for score in transformed_importance_scores])

plt.figure(figsize=(10, 6))
sns.histplot(all_scores, bins=50, kde=True)
plt.title('Distribution of All Transformed Importance Scores')
plt.xlabel('Transformed Importance Score')
plt.ylabel('Frequency')
plt.grid()
plt.show()




In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from matplotlib.colors import TwoSlopeNorm
from matplotlib import cm

class GradCAM:
    """
    Implements Gradient-weighted Class Activation Mapping for model interpretation.
    """
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        
        # Register hooks to capture activations and gradients
        self.register_hooks()
        
    def register_hooks(self):
        def forward_hook(module, input, output):
            # Store the activations of the target layer
            self.activations = output.detach()
            
        def backward_hook(module, grad_input, grad_output):
            # Store the gradients coming into the target layer
            self.gradients = grad_output[0].detach()
        
        # Register the hooks
        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_backward_hook(backward_hook)
        
    def generate_ais(self, input_image):
        # Forward pass through the model
        model_output, _ = self.model(input_image)
        
        # Clear previous gradients
        self.model.zero_grad()
        
        # Backward pass - for regression we use the output directly
        model_output.backward(retain_graph=True)
        
        # Get the gradients and activations
        gradients = self.gradients.data.cpu().numpy()[0]  # [C, H, W]
        activations = self.activations.data.cpu().numpy()[0]  # [C, H, W]
        
        # Weight the channels by the average gradient
        weights = np.mean(gradients, axis=(1, 2))  # [C]
        
        # Create weighted combination of activation maps
        cam = np.zeros(activations.shape[1:], dtype=np.float32)  # [H, W]
        for i, w in enumerate(weights):
            cam += w * activations[i, :, :]
        
        # Apply ReLU to focus on features that have a positive influence
        cam = np.maximum(cam, 0)
        
        # Resize CAM to input image size
        cam = cv2.resize(cam, (input_image.shape[2], input_image.shape[3]))
        
        # Normalize the CAM
        cam = cam - np.min(cam)
        cam = cam / (np.max(cam) + 1e-7)  # Adding small constant to avoid division by zero
        
        return cam

class AIS:
    """
    Implements CAM for model interpretation.
    """
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        
        self.activations = None
        
        # Register hooks to capture activations and gradients
        self.register_hooks()
        
    def register_hooks(self):
        def forward_hook(module, input, output):
            # Store the activations of the target layer
            self.activations = output.detach()
            
        # Register the hooks
        self.target_layer.register_forward_hook(forward_hook)
        
    def generate_ais(self, input_image, importance_scores=None):
        # Forward pass through the model
        model_output, _ = self.model(input_image)
        
        # Get the activations
        activations = self.activations.data.cpu().numpy()[0]  # [C, H, W]
        
        # Relu the activations to focus on positive influences
        activations = np.maximum(activations, 0)
        
        # Weight the channels by the importance scores
        weights = importance_scores  # Use the instance variable
            
        # Create weighted combination of activation maps
        ais = np.zeros(activations.shape[1:], dtype=np.float32)  # [H, W]
        
        # Handle the case where weights might be multi-dimensional
        if len(weights.shape) == 1 and len(weights) == activations.shape[0]:
            # Standard case: one weight per channel
            for i, w in enumerate(weights):
                ais += w * activations[i, :, :]
        else:
            # Check the shapes before operation to provide helpful error message
            raise ValueError(f"Incompatible shapes: weights {weights.shape}, activations {activations.shape}")
        
        # Apply ReLU to focus on features that have a positive influence
        # ais = np.maximum(ais, 0)
        
        # Resize ais to input image size
        ais = cv2.resize(ais, (input_image.shape[2], input_image.shape[3]))
        
        # Normalize the ais
        max_abs_val = max(abs(np.min(ais)), abs(np.max(ais)))
        if max_abs_val > 0:  # Avoid division by zero
            ais = ais / max_abs_val
        
        return ais

def apply_gradcam_to_dataset(model, dataloader, target_layer, output_dir, model_name, device, num_samples=10):
    """
    Apply Grad-CAM visualization to a subset of images from the dataset.
    
    Args:
        model: Neural network model
        dataloader: DataLoader containing the images
        target_layer: Target layer for Grad-CAM (usually the last convolutional layer)
        output_dir: Directory to save the visualizations
        model_name: Name of the model for saving files
        device: Device to run the model on (cuda/cpu)
        num_samples: Number of samples to visualize
    """
    # Create model-specific output directory
    model_output_dir = os.path.join(output_dir, model_name)
    os.makedirs(model_output_dir, exist_ok=True)
    
    # Initialize Grad-CAM with the model and target layer
    grad_cam = GradCAM(model, target_layer)
    
    # Move model to device and set to evaluation mode
    model.to(device)
    model.eval()
    
    # Process images from the dataloader
    samples_processed = 0
    
    for batch_idx, (images, labels) in enumerate(dataloader):
        if samples_processed >= num_samples:
            break
            
        images = images.to(device)
        labels = labels.to(device)
        
        for i in range(images.shape[0]):
            if samples_processed >= num_samples:
                break
                
            # Get the single image
            image = images[i:i+1]
            image.requires_grad = True  # Enable gradients for this image
            
            # Generate the Grad-CAM visualization
            cam = grad_cam.generate_ais(image)
            
            # Convert to heatmap using jet colormap
            heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
            
            # Prepare original image for visualization
            img_tensor = images[i].cpu().numpy()
            img_tensor = np.transpose(img_tensor, (1, 2, 0))  # [H, W, C]
            
            # Denormalize the image
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            img_tensor = std * img_tensor + mean
            img_tensor = np.clip(img_tensor, 0, 1)
            
            # Convert to uint8 for OpenCV
            rgb_img = (img_tensor * 255).astype(np.uint8)
            bgr_img = rgb_img[:, :, ::-1]  # RGB to BGR for OpenCV
            
            # Resize heatmap to match the image size
            heatmap = cv2.resize(heatmap, (bgr_img.shape[1], bgr_img.shape[0]))
            
            # Create overlay of heatmap on original image
            overlay = cv2.addWeighted(bgr_img, 0.6, heatmap, 0.4, 0)
            
            # Save original image
            plt.figure()
            plt.imshow(rgb_img)
            plt.axis('off')
            save_path = os.path.join(model_output_dir,'original_images',f"original_{samples_processed}.png")
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            plt.savefig(save_path, bbox_inches='tight', dpi=150)
            plt.close()
            
            # Save Grad-CAM heatmap
            plt.figure()
            plt.imshow(cam, cmap='jet')
            plt.axis('off')
            save_path = os.path.join(model_output_dir, 'raw_heatmap', f"heatmap_{samples_processed}.png")
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            plt.savefig(save_path, bbox_inches='tight', dpi=150)
            plt.close()
            
            # Save overlay
            plt.figure()
            plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))

            plt.axis('off')
            save_path = os.path.join(model_output_dir, 'overlay' , f"overlay_{samples_processed}.png")
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            plt.savefig(save_path, bbox_inches='tight', dpi=150)
            plt.close()
            
            samples_processed += 1
            print(f"Processed {samples_processed}/{num_samples} samples for {model_name} model")
        
def apply_redblue_ais_to_dataset(model, dataloader, target_layer, output_dir, model_name, device, importance_scores, num_samples=10):
    """
    Apply AIS visualization to a subset of images from the dataset with a red-blue colormap.
    
    Args:
        model: Neural network model
        dataloader: DataLoader containing the images
        target_layer: Target layer for AIS (usually the last convolutional layer)
        output_dir: Directory to save the visualizations
        model_name: Name of the model for saving files
        device: Device to run the model on (cuda/cpu)
        importance_scores: Importance weights for each channel in the target layer
                          (Can contain positive and negative values)
        num_samples: Number of samples to visualize
    """
    # Create model-specific output directory
    model_output_dir = os.path.join(output_dir, model_name)
    os.makedirs(model_output_dir, exist_ok=True)
    
    # Initialize AIS with the model and target layer
    ais_generator = AIS(model, target_layer)
    
    # Move model to device and set to evaluation mode
    model.to(device)
    model.eval()
    
    # Process images from the dataloader
    samples_processed = 0
    
    for batch_idx, (images, labels) in enumerate(dataloader):
        if samples_processed >= num_samples:
            break
            
        images = images.to(device)
        labels = labels.to(device)
        
        for i in range(images.shape[0]):
            if samples_processed >= num_samples:
                break
                
            # Get the single image
            image = images[i:i+1]

            # Determine which importance scores to use based on shape
            if len(importance_scores[i].shape) == 1 and len(importance_scores[i]) == images.shape[1]:
                # Use the importance scores directly
                raw_heatmap = ais_generator.generate_ais(image, importance_scores)
            else:
                # Use the index to access the corresponding importance scores
                raw_heatmap = ais_generator.generate_ais(image, importance_scores[i])

            

            # We'll keep the raw heatmap values in the [-1, 1] range without normalization to [0, 1]
            # This preserves the sign information we need for the red-blue visualization
            
            # Prepare original image for visualization
            img_tensor = images[i].cpu().numpy()
            img_tensor = np.transpose(img_tensor, (1, 2, 0))  # [H, W, C]
            
            # Denormalize the image
            mean = np.array([0.485, 0.456, 0.406])
            std = np.array([0.229, 0.224, 0.225])
            img_tensor = std * img_tensor + mean
            img_tensor = np.clip(img_tensor, 0, 1)
            
            # Convert to uint8 for OpenCV
            rgb_img = (img_tensor * 255).astype(np.uint8)
            
            # Save original image
            plt.figure()
            plt.imshow(rgb_img)
            plt.axis('off')
            save_path = os.path.join(model_output_dir, 'original_images', f"original_{samples_processed}.png")
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            plt.savefig(save_path, bbox_inches='tight', dpi=150)
            plt.close()
            
            # Create the red-blue heatmap using the coolwarm colormap
            plt.figure()
            # Create a normalization that's centered at zero
            norm = TwoSlopeNorm(vmin=-1, vcenter=0, vmax=1)
            # Apply the coolwarm colormap (red for positive, blue for negative)
            cmap = cm.get_cmap('seismic')
            plt.imshow(raw_heatmap, cmap=cmap, norm=norm)
            plt.colorbar(label='Contribution to Realism')
            plt.title("Red: Positive Contribution, Blue: Negative Contribution")
            plt.axis('off')
            save_path = os.path.join(model_output_dir, 'raw_heatmap', f"heatmap_{samples_processed}.png")
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            plt.savefig(save_path, bbox_inches='tight', dpi=150)
            plt.close()
            
            # Create overlay of heatmap on original image
            # First, convert the raw heatmap to a colored image using matplotlib
            plt.figure(figsize=(10, 8))
            plt.imshow(raw_heatmap, cmap=cmap, norm=norm)
            plt.axis('off')
            # Save to a temporary file
            temp_path = os.path.join(model_output_dir, 'temp', f"temp_heatmap_{samples_processed}.png")
            os.makedirs(os.path.dirname(temp_path), exist_ok=True)
            plt.savefig(temp_path, bbox_inches='tight', pad_inches=0, dpi=150)
            plt.close()
            
            # Read the colored heatmap with OpenCV
            colored_heatmap = cv2.imread(temp_path)
            # Resize heatmap to match the image size
            colored_heatmap = cv2.resize(colored_heatmap, (rgb_img.shape[1], rgb_img.shape[0]))
            
            # Create overlay of heatmap on original image
            # We need to convert RGB to BGR for OpenCV
            bgr_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2BGR)
            overlay = cv2.addWeighted(bgr_img, 0.6, colored_heatmap, 0.4, 0)
            
            # Save overlay
            plt.figure()
            plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            save_path = os.path.join(model_output_dir, 'overlay', f"overlay_{samples_processed}.png")
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            plt.savefig(save_path, bbox_inches='tight', dpi=150)
            plt.close()
            
            # Remove temporary file
            if os.path.exists(temp_path):
                os.remove(temp_path)
            
            samples_processed += 1
            print(f"Processed {samples_processed}/{num_samples} samples for {model_name} model")
   

In [ ]:
# Define output directory for Grad-CAM visualizations
OUTPUT_DIR_GRADCAM = 'Heatmap_images/GradCAM'
OUTPUT_DIR_CAM = 'Heatmap_images/Dual_scores_OBJ_X_OBJ_CAM'
os.makedirs(OUTPUT_DIR_GRADCAM, exist_ok=True)
os.makedirs(OUTPUT_DIR_CAM, exist_ok=True)

# For VGG16-based models, the last convolutional layer is usually best for Grad-CAM
target_layer_baseline = baseline_model.features[28]
target_layer_noisy_pruned = noisy_pruned_model.features[28]

# Define number of samples to visualize
NUM_SAMPLES = len(dataloaders['test'].dataset)

# Apply CAM to noisy pruned model
print("Generating CAM visualizations for baseline model with importance scores...")
apply_redblue_ais_to_dataset(
    model=baseline_model,
    dataloader=dataloaders['test'],
    target_layer=target_layer_baseline,
    output_dir=OUTPUT_DIR_CAM,
    model_name='Baseline with importance scores',
    device=device,
    importance_scores=transformed_importance_scores,
    num_samples=NUM_SAMPLES
)

# # Apply CAM to baseline model
# print("Generating CAM visualizations for baseline model...")
# apply_cam_to_dataset(
#     model=baseline_model,
#     dataloader=dataloaders['test'],
#     target_layer=target_layer_baseline,
#     output_dir=OUTPUT_DIR_CAM,
#     model_name='baseline with uniform importance scores',
#     device=device,
#     importance_scores=np.ones(512),  # Use uniform importance scores for baseline model
#     num_samples=NUM_SAMPLES
# )

# # Apply Grad-CAM to baseline model
# print("Generating Grad-CAM visualizations for baseline model...")
# apply_gradcam_to_dataset(
#     model=baseline_model,
#     dataloader=dataloaders['test'],
#     target_layer=target_layer_baseline,
#     output_dir=OUTPUT_DIR_GRADCAM,
#     model_name='baseline',
#     device=device,
#     num_samples=NUM_SAMPLES
# )

# # Apply Grad-CAM to noisy pruned model
# print("Generating Grad-CAM visualizations for noisy pruned model...")
# apply_gradcam_to_dataset(
#     model=noisy_pruned_model,
#     dataloader=dataloaders['test'],
#     target_layer=target_layer_noisy_pruned,
#     output_dir=OUTPUT_DIR_GRADCAM,
#     model_name='noisy_pruned',
#     device=device,
#     num_samples=NUM_SAMPLES
# )



